# Ansbile playbook

## 1. Simple playbook

### 1.1. Run playbook with current user

In [ ]:
ansible-playbook ping.yml

### 1.2. Run playbook with become user

In [77]:
ansible-playbook ping_root.yml --vault-id vault-id -e "@password.yml"


PLAY [Test availability of remote server] **************************************

TASK [Gathering Facts] *********************************************************
ok: [debian1]

TASK [Ping remote server] ******************************************************
ok: [debian1]

TASK [Show ping result] ********************************************************
ok: [debian1] => {
    "msg": "pong"
}

PLAY RECAP *********************************************************************
debian1                    : ok=3    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



### 1.3. Show ansible variables

#### 1.3.1. Show simple variables

In [78]:
ENV_VAR="HELLO WORLD" ansible-playbook vars.yml


PLAY [Test ansible variables] **************************************************

TASK [Gathering Facts] *********************************************************
ok: [debian1]

TASK [Show some ansible variables] *********************************************
ok: [debian1] => {
    "msg": [
        "Default IPV4: {'gateway': '10.0.2.2', 'interface': 'eth0', 'address': '10.0.2.15', 'broadcast': '10.0.2.255', 'netmask': '255.255.255.0', 'network': '10.0.2.0', 'macaddress': '08:00:27:8d:c0:4d', 'mtu': 1500, 'type': 'ether', 'alias': 'eth0'}",
        "Architecture: x86_64",
        "Distribution: Debian"
    ]
}

TASK [Show user define variables] **********************************************
ok: [debian1] => {
    "msg": [
        "User name: Alvin",
        "Home page: https://www.alvin.edu/ansible/playbook",
        "Variable A: Unknown",
        "Variable B: Unknown"
    ]
}

TASK [Include vars from file] **************************************************
ok: [debian1]

TASK [Show use

#### 1.3.2. Show all variables

In [ ]:
SHOW_ALL=true ENV_VAR="HELLO WORLD" ansible-playbook vars.yml

### 1.3. Conditions

#### 1.3.1. Basic conditionals with `when`

In [ ]:
ansible-playbook conditions.yml --tags T1

#### 1.3.2. Conditionals based on `ansible_facts`

In [ ]:
ansible-playbook conditions.yml --tags T2